In [3]:
from ipycytoscape import CytoscapeWidget
import networkx as nx
from ipycytoscape import Node, Edge
import rdflib
from ipyradiant import FileManager, PathLoader
from ipyradiant.rdf2nx import URItoID, URItoShortID
from rdflib.namespace import RDFS
from ipyradiant.rdf2nx import RDF2NX
import json
import ipycytoscape
import ipywidgets as W
from IPython.display import JSON, display

Set up graph using stuff from RDF_to_NX.ipynb

In [4]:
lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX hum: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (hum:1)  # Luke
            (hum:4)  # Vader
            (film:1) # A New Hope
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))
uri = RDFS.label
ns = {"rdfs": str(RDFS)}
initNs = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "res": "https://swapi.co/resource/",
    "voc": "https://swapi.co/vocabulary/",
    "base": "https://swapi.co/resource/",
}
 
nx_graph = RDF2NX.convert(simple_graph, namespaces=initNs)
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(
        node.data, indent=2
    )

Update layout.

Create HBox with buttons, json viewer

In [5]:
expand_button = W.Button(description="Expand Upon Selected Node", layout = W.Layout(width='250px', height='40px'))
remove_button = W.Button(description="Remove Expansions Upon Node", layout = W.Layout(width='250px', height='40px'))
left_vbox = W.VBox(children=[directed, W.HBox(children=[expand_button,remove_button])], border=True, layout=W.Layout(height='450px', border='solid 2px'))
json_output = W.Output()
right_vbox = W.VBox(children=[W.HTML("<h1>JSON Data from Node</h1>", layout=W.Layout(height='100px')), json_output])
full_display = W.GridBox(children = [left_vbox, right_vbox],layout=W.Layout(grid_template_columns="repeat(2, 500px)", height='450px') )

In [6]:
def log_node_clicks(node):
    full_display.selected_node = node #['data']['iri'] ??? do i want this
    data = node['data']
    data.pop('_label', None)
    data.pop('_attrs', None)
    with json_output:
        json_output.clear_output()
        display(JSON(data))
    
directed.on("node", "click", log_node_clicks)
directed.set_layout(
    name="dagre", animate=False, randomize=False, maxSimulationTime=2000
)
# Workaround for style overwriting
directed.set_style(
    [
        {
            "selector": "node",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "blue",
            },
        },
        {
            "selector": "edge",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "green",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
            },
        },
        {"selector": "edge.multiple_edges", "style": {"curve-style": "bezier"}},
        {
            "selector": ":active ",
            "css": {
                "label": "data(_attrs)",
                "text-wrap": "wrap",
                "text-max-width": "500px",
                "text-valign": "bottom",
                "text-halign": "right",
                "text-background-opacity": 0.9,
                "text-background-color": "white",
                "text-background-shape": "roundrectangle",
                "color": "black",
            },
        },
    ]
)

In [7]:
full_display

GridBox(children=(VBox(children=(CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'animate': False, 'randomi…

## Now attempting to put this into a class.

In [15]:
import traitlets as trt
from ipyradiant.query.api import SPARQLQueryFramer
import pandas 
import rdflib
from typing import Union
from rdflib.namespace import RDF


class GetOutgoingPredicateObjects(SPARQLQueryFramer):
    sparql =  """
    SELECT DISTINCT ?s ?p ?o ?label
    
    WHERE {
        ?s ?p ?o .
        FILTER (!isLiteral(?o))
        OPTIONAL {?o rdfs:label ?label}
    }
    
    """
    @classmethod
    def run_query(
        cls,
        graph: rdflib.graph.Graph,
        subject: Union[rdflib.term.URIRef, str],
        initBindings: dict = None,
        **initBindingsKwarg,
    ) -> pandas.DataFrame:
        """Overwrite the super method in order to wrap with validation checks."""
        
        qres = super().run_query(graph, s=rdflib.term.URIRef(subject), initBindings=initBindings, **initBindingsKwarg)
        # Validating with known requirements on query results
        # TODO
        return qres


class InteractiveViewer(W.VBox):
    expand_button = trt.Instance(W.Button)
    remove_button = trt.Instance(W.Button)
    cyto_graph = trt.Instance(ipycytoscape.CytoscapeWidget)
    selected_node = trt.Dict(allow_none=True)
    rdf_graph = trt.Instance(rdflib.graph.Graph, allow_none=True)
    cyto_style = trt.List(allow_none=True)
    
    # TODO: should this be a traitlet?
    _uri_converter = URItoShortID
    
    
    @trt.default("expand_button")
    def _create_expand_button(self):
        return W.Button(
            description="Expand Upon Selected Node", 
            layout = W.Layout(width='500px', height='40px')
        )
    
    @trt.default("remove_button")
    def _create_remove_button(self):
        return W.Button(
            description="Remove Expansions Upon Node", 
            layout = W.Layout(width='500px', height='40px')
        )
    
    @trt.default("selected_node")
    def _create_default_selected_node(self):
        return None
    
    @trt.default("cyto_style")
    def _create_cyto_style(self):
        return  [
        {
            "selector": "node",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "blue",
            },
        },
        {
            "selector": "node[classes='selected-node']",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "background-color": "red",
            },
        },
        {
            "selector": "node[classes='temp-node']",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "background-color": "green",
            },
        },
        {
            "selector": "edge[classes='temp-edge']",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "green",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
            },
        },
        {"selector": "edge.multiple_edges", "style": {"curve-style": "bezier"}},
    ]
    
    
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if kwargs is not None and 'rdf_graph' in kwargs:
            self.rdf_graph = kwargs['rdf_graph']
        else:
            print('must pass in an rdf_graph')
        if kwargs is not None and 'cyto_graph' in kwargs:
            self.cyto_graph = kwargs['cyto_graph']
            self.cyto_graph.set_layout(
                    name="dagre", animate=False, randomize=False, maxSimulationTime=2000
            )
        else:
            print('must pass in a cyto_graph')

        
        self.cyto_graph.set_style(self.cyto_style)
        self.expand_button.on_click(self.expand_button_clicked)
        self.cyto_graph.on("node", "click", self.log_node_clicks)
        self.buttons = W.HBox(children = [self.expand_button, self.remove_button])
        self.children = [self.cyto_graph, self.buttons]
        self.layout = W.Layout(width='1000px', border='solid 2px')
    
    
    def log_node_clicks(self, node):
        try:
            self.selected_node['data']['classes'] = None
        except:
            pass
        self.selected_node = node
        self.selected_node['data']['classes'] = 'selected-node' 
        data = node['data']
        # TODO: replace pops with filter for private attributes
        data.pop('_label', None)
        data.pop('_attrs', None)
        with json_output:
            json_output.clear_output()
            display(JSON(data))
        
    def expand_button_clicked(self, b):
        if self.selected_node is None:
            return None
        new_data = GetOutgoingPredicateObjects.run_query(graph=self.rdf_graph, subject=self.selected_node['data']['iri'])
        objs = new_data['o'].tolist()
        preds = new_data['p'].tolist()
        labels = new_data['label'].tolist()
        # add nodes
        for ii, x in enumerate(objs):
            print(str(x))
            if preds[ii] == RDF.type:
                labels[ii] = "Type: " + self._uri_converter(str(x))
            self.cyto_graph.graph.add_node(Node(data={'id':str(x), 'classes':'temp-node', '_label':labels[ii]}))
            self.cyto_graph.graph.add_edge(Edge(data={'source':self.selected_node['data']['id'], 'target': str(x), 'classes':'temp-edge'}))


            
        
        
        
    
    
    
    
    
    

In [22]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)
for node in directed.graph.nodes:
    # deal with inability to handle colons
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(
        node.data, indent=2
    )

In [23]:
iv = InteractiveViewer(rdf_graph=rdf_graph, cyto_graph=directed)
combined = W.GridBox(children = [iv, json_output],layout=W.Layout(grid_template_columns="repeat(2, 500px)", height='450px') )

In [24]:
combined

GridBox(children=(InteractiveViewer(children=(CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'animate': Fa…

## TODO for Next Meeting/Next week:

- Edge Styling?
- Mark the selected node
- Remove Edges
- Integrate more into the class, less work before
- Best way to get labels on temp nodes?

In [110]:
iv.selected_node

{'data': {'id': 'https://swapi.co/resource/film/6',
  'classes': 'selected-node'},
 'position': {'x': 415, 'y': 95},
 'group': 'nodes',
 'removed': False,
 'selected': False,
 'selectable': False,
 'locked': False,
 'grabbable': True,
 'pannable': False,
 'classes': ''}

In [ ]:
%debug

In [ ]:
%debug

In [ ]:
%debug